# Homework 3 - Credit Analytics
Submission by Dennis Goldenberg

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import itertools
from sklearn.decomposition import PCA
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import datetime as dt

## $\underline{\text{Question 1}}$

### a. Examining volatility and correlation structure of yield curves
I read the yield curve in and generate the daily percent change Matrix:

In [79]:
ticker = ['DGS1MO',  'DGS3MO', 'DGS6MO','DGS1','DGS2',
           'DGS3', 'DGS5','DGS7', 'DGS10','DGS20','DGS30']
sdt = dt.datetime(2003, 4, 10)
edt = dt.datetime(2024, 4, 12)
source = 'fred'
yieldcurve = pd.DataFrame(web.DataReader(ticker, source, sdt, edt))
yieldcurve = yieldcurve.dropna()
pctChange = yieldcurve.pct_change().dropna()
pctChange

,DGS1MO,DGS3MO,DGS6MO,DGS1,DGS2,DGS3,DGS5,DGS7,DGS10,DGS20,DGS30
DATE,,,,,,,,,,,
2003-04-11,0.000000,0.026316,0.025862,0.040000,0.031250,0.029557,0.020761,0.008671,0.012658,0.004049,0.003868
2003-04-14,0.008475,0.025641,0.025210,0.023077,0.054545,0.038278,0.023729,0.017192,0.010000,0.006048,0.001927
2003-04-15,-0.016807,-0.016667,-0.008197,-0.007519,-0.028736,-0.023041,-0.019868,-0.016901,-0.014851,-0.012024,-0.001923
2003-04-16,-0.017094,-0.008475,-0.008264,-0.007576,-0.011834,-0.009434,-0.003378,-0.002865,-0.005025,-0.002028,-0.011561
2003-04-17,-0.008696,0.008547,0.000000,0.022901,0.023952,0.023810,0.013559,0.005747,0.005051,-0.002033,-0.001949
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-05,0.000000,0.003697,0.003759,0.010000,0.017204,0.017937,0.018605,0.018561,0.018561,0.017505,0.015660
2024-04-08,0.001828,0.000000,0.001873,0.003960,0.010571,0.013216,0.011416,0.009112,0.006834,0.000000,0.002203
2024-04-09,0.000000,0.000000,-0.001869,-0.007890,-0.008368,-0.017391,-0.013544,-0.011287,-0.013575,-0.010753,-0.010989


One slight issue is that, for maturities of 1 and 3 months, there are cases where the yield is 0, meaning the next day's percentage change would be infinity. I generate the percent change matrix with these rows excluded to get sensible results:

In [80]:
mask1MO = ((pctChange["DGS1MO"] > 5000000) | (pctChange["DGS1MO"] < -5000000)).values
mask3MO = ((pctChange["DGS3MO"] > 5000000) | (pctChange["DGS3MO"] < -5000000)).values
mask = mask1MO | mask3MO
pCNoInf = pctChange[~(mask)]

I examine both the means and standard deviations of the percent changes of the individual maturities, as well as the entire variance-covariance matrix:

In [113]:
meansSD = np.vstack([pCNoInf.mean(), pCNoInf.std()])
meanSDChange = pd.DataFrame(meansSD.T, index = pCNoInf.columns,
                            columns = ["Average % change","Standard Deviation"])
meanSDChange

,Average % change,Standard Deviation
DGS1MO,0.026163,0.340905
DGS3MO,0.017465,0.257916
DGS6MO,0.004204,0.106689
DGS1,0.001608,0.052364
DGS2,0.001433,0.049136
DGS3,0.001131,0.043147
DGS5,0.000775,0.035401
DGS7,0.000534,0.029102
DGS10,0.000406,0.025360
DGS20,0.000231,0.019903


In [93]:
pCNoInf.corr()

,DGS1MO,DGS3MO,DGS6MO,DGS1,DGS2,DGS3,DGS5,DGS7,DGS10,DGS20,DGS30
DGS1MO,1.000000,0.286878,0.193645,0.145985,0.046643,0.045682,0.026222,0.019487,0.016348,0.022277,0.022446
DGS3MO,0.286878,1.000000,0.354084,0.186800,0.097013,0.083241,0.066039,0.056480,0.054318,0.050517,0.052287
DGS6MO,0.193645,0.354084,1.000000,0.285862,0.145923,0.141133,0.124739,0.109270,0.116977,0.120389,0.121915
DGS1,0.145985,0.186800,0.285862,1.000000,0.457494,0.451110,0.407452,0.385453,0.354372,0.320482,0.306776
DGS2,0.046643,0.097013,0.145923,0.457494,1.000000,0.807510,0.761876,0.701888,0.630248,0.547689,0.507561
DGS3,0.045682,0.083241,0.141133,0.451110,0.807510,1.000000,0.914177,0.859409,0.790405,0.704853,0.656172
DGS5,0.026222,0.066039,0.124739,0.407452,0.761876,0.914177,1.000000,0.959264,0.906662,0.828131,0.781175
DGS7,0.019487,0.056480,0.109270,0.385453,0.701888,0.859409,0.959264,1.000000,0.962748,0.903305,0.864194
DGS10,0.016348,0.054318,0.116977,0.354372,0.630248,0.790405,0.906662,0.962748,1.000000,0.957604,0.929268
DGS20,0.022277,0.050517,0.120389,0.320482,0.547689,0.704853,0.828131,0.903305,0.957604,1.000000,0.972051


I make 3 observations from these two results:
- The average percent change and standard deviation of percent change is $\textbf{higher}$ for shorter maturities, and $\textbf{lower}$ for longer maturities. This suggests that treasury rates with shorter maturities are more volatile than treasuries with longer maturities.
- All entries in the correlation matrix are positive. This suggests that there is a common market factor moving yields of all maturities in one direction or the other.
- For any given maturity length, the change in yield of that maturity length is generally more correlated with changes in yields of similar maturity lengths. For example, change in yield of Treasury securities with a maturity of 30 years is most correlated with the change in securities with maturities of 10 and 20 years.

### b. Estimating $\alpha, \beta, \sigma$ in simple short rate model

### c. Calculating spread between points with one month yield

### d. Monte Carlo Simulation Engine
Read "Some tricks on Monte Carlo Simulation"

### e. Pricing Interest Rate Swap

### f. Potential future exposure